In [1]:
import numpy as np
import pandas as pd
import torch
import random

random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
train = pd.read_csv('mnist_train.csv')
test = pd.read_csv('mnist_test.csv')
submit = pd.read_csv('submission.csv')

In [3]:
train

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,784
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
59996,59996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
59997,59997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
59998,59998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [4]:
train_x= train.drop(['Unnamed: 0', '784'],axis =1)
test_x = test.drop(['Unnamed: 0'], axis = 1)
train_y = train['784']

In [5]:
train_x_tensor = torch.FloatTensor(train_x.values).to(device)
train_y_tensor = torch.LongTensor(train_y.values).to(device)

test_x_tensor = torch.FloatTensor(test_x.values).to(device)

In [6]:
train_dataset = torch.utils.data.TensorDataset(train_x_tensor, train_y_tensor)
data_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 1000, shuffle = True, drop_last = True)

In [ ]:
#train_dataset = torch.utils.data.TensorDataset(train_x_tensor, train_y_tensor)
#dataloader = torch.utils.data.DataLoader(dataset = train_dataset , batch_size = 1000, shuffle = True, drop_last = True)


모델만들기

In [7]:
linear1 = torch.nn.Linear(784, 512, bias= True )
linear2 = torch.nn.Linear(512, 512, bias = True)
linear3 = torch.nn.Linear(512, 10, bias = True)
relu = torch.nn.ReLU()
sigmoid = torch.nn.Sigmoid()
dropout = torch.nn.Dropout(p=0.3)

In [8]:
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)

Parameter containing:
tensor([[ 0.0885,  0.0083, -0.0234,  ..., -0.0318,  0.0232, -0.1198],
        [-0.0533,  0.0282,  0.0466,  ..., -0.0937,  0.0293,  0.0587],
        [-0.0752,  0.0002, -0.0379,  ...,  0.0974,  0.0178,  0.0756],
        ...,
        [ 0.0516, -0.0587,  0.0222,  ...,  0.0495,  0.1053, -0.0017],
        [-0.0122, -0.0071, -0.0201,  ...,  0.0811,  0.1085, -0.1117],
        [-0.0658,  0.0600, -0.0701,  ...,  0.0131,  0.1194, -0.0814]],
       requires_grad=True)

In [9]:
model = torch.nn.Sequential(linear1, relu, dropout, linear2, relu, dropout, linear3).to(device)

In [10]:
loss = torch.nn.CrossEntropyLoss().to(device) #회귀 MSELoss 이진분류 L1Loss
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

train 구간

In [11]:
total_epoch = 100
model.train()
total_batch = len(data_loader)
for epoch in range(1,total_epoch+1):
    avg_cost = 0
    for X, Y in data_loader:
        optimizer.zero_grad()
        
        hypothesis = model(X)
        cost = loss(hypothesis,Y)
        cost.backward()
        
        optimizer.step()
        
        avg_cost += cost/total_batch
    if epoch % 10 ==0:
        print(epoch, avg_cost)
        

10 tensor(0.2660, device='cuda:0', grad_fn=<AddBackward0>)
20 tensor(0.1190, device='cuda:0', grad_fn=<AddBackward0>)
30 tensor(0.0697, device='cuda:0', grad_fn=<AddBackward0>)
40 tensor(0.0561, device='cuda:0', grad_fn=<AddBackward0>)
50 tensor(0.0502, device='cuda:0', grad_fn=<AddBackward0>)
60 tensor(0.0473, device='cuda:0', grad_fn=<AddBackward0>)
70 tensor(0.0433, device='cuda:0', grad_fn=<AddBackward0>)
80 tensor(0.0449, device='cuda:0', grad_fn=<AddBackward0>)
90 tensor(0.0371, device='cuda:0', grad_fn=<AddBackward0>)
100 tensor(0.0366, device='cuda:0', grad_fn=<AddBackward0>)


In [12]:
with torch.no_grad():
    model.eval()
    prediction = model(test_x_tensor)
    
    predict = torch.argmax(prediction, dim = 1)

In [15]:
#이진 분류인 경우 L1Loss
#predict = predcition.detach().cpu().numpy()
#submit['Label'] = (predict>0.5).astype('int')

#회귀인 경우 MSEloss
#predict = prediction.detach().cpu().numpy()

In [13]:
submit['Label']=predict.detach().cpu().numpy()

In [ ]:
submit.to_csv('memorize_submit.csv', index = False)